In [23]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [24]:
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [26]:
##Here we are trying to predit the output which is 'time' but it is in category datatype so we are doing to
#LabelEncoding to convet object type into numerical types

from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [27]:
df['time'].value_counts()

0    176
1     68
Name: time, dtype: int64

In [28]:
#dividing the data into dependent and independent
x=df.drop(labels=['time'],axis=1)
y=df['time']

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=17)

In [31]:
#@title Steps to be carried out in ML predictiom
## Handling Missing Values
## handling Categorical features
## handling outliers
## Feature scaling
## Automating the entire

In [33]:
from sklearn.impute import SimpleImputer ## Handling Missing Values
from sklearn.preprocessing import OneHotEncoder## handling Categorical features
from sklearn.preprocessing import StandardScaler## Feature scaling
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
## Automating the entire

In [34]:
categorical_cols = ['sex', 'smoker','day']
numerical_cols = ['total_bill', 'tip','size']

In [35]:
## Feature Engineering Automation
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')), ##missing values
        ('scaler',StandardScaler())## feature scaling
    ]

)

#categorical Pipeline
cat_pipeline=Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='most_frequent')), ## handling Missing values
                ('onehotencoder',OneHotEncoder()) ## Categorical features to numerical
                ]

            )


In [36]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)

])

In [38]:
x_train=preprocessor.fit_transform(x_train)
x_test=preprocessor.transform(x_test)

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [40]:
## Automate Model Training Process
models={
    'Random Forest':RandomForestClassifier(),
    'Decision Tree':DecisionTreeClassifier(),
    'SVC':SVC()

}

In [41]:
from sklearn.metrics import accuracy_score

In [42]:
def evaluate_model(X_train,y_train,X_test,y_test,models):

    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        # Train model
        model.fit(X_train,y_train)



        # Predict Testing data
        y_test_pred =model.predict(X_test)

        # Get accuracy for test data prediction

        test_model_score = accuracy_score(y_test,y_test_pred)

        report[list(models.keys())[i]] =  test_model_score



    return report

In [44]:
evaluate_model(x_train,y_train,x_test,y_test,models)

{'Random Forest': 0.9508196721311475,
 'Decision Tree': 0.9180327868852459,
 'SVC': 0.9508196721311475}

In [45]:
classifier=RandomForestClassifier()

In [46]:
## Hypeparameter Tuning
params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
              }

In [47]:
from sklearn.model_selection import RandomizedSearchCV

In [49]:
cv=RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)
cv.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=5, n_estimators=300;, score=0.946 total time=   0.5s
[CV 2/5] END criterion=entropy, max_depth=5, n_estimators=300;, score=0.973 total time=   0.5s
[CV 3/5] END criterion=entropy, max_depth=5, n_estimators=300;, score=0.946 total time=   0.5s
[CV 4/5] END criterion=entropy, max_depth=5, n_estimators=300;, score=0.972 total time=   0.5s
[CV 5/5] END criterion=entropy, max_depth=5, n_estimators=300;, score=0.944 total time=   0.5s
[CV 1/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.946 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.946 total time=   0.2s
[CV 3/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.946 total time=   0.3s
[CV 4/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=1.000 total time=   0.3s
[CV 5/5] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.972 total ti

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [50]:
cv.best_params_

{'n_estimators': 100, 'max_depth': 10, 'criterion': 'entropy'}